In [2]:
import requests
import os
from PyPDF4 import PdfFileReader
import tabula


In [3]:
def download_file(url, output_folder):
    response = requests.get(url)
    file_name = url.split('/')[-1]
    output_path = os.path.join(output_folder, file_name)

    with open(output_path, 'wb') as f:
        f.write(response.content)
    return output_path


def find_keyword_pages(pdf_path, keyword):
    with open(pdf_path, 'rb') as f:
        reader = PdfFileReader(f)
        num_pages = reader.getNumPages()

        pages_with_keyword = []

        for page in range(num_pages):
            text = reader.getPage(page).extractText()
            if keyword in text:
                pages_with_keyword.append(page + 1)

    return pages_with_keyword


def extract_and_save_tables(pdf_path, pages, keyword, output_folder):
    for page in pages:
        tables = tabula.read_pdf(pdf_path, pages=page, multiple_tables=True)
        for i, table in enumerate(tables):
            output_path = os.path.join(
                output_folder, f"{keyword}_table_page{page}_{i + 1}.csv")
            table.to_csv(output_path, index=False)


In [4]:
output_folder = "asset/"
pdf_paths = ["asset/pdf/abl여30세.pdf",
             "asset/pdf/LKB.pdf",
             "asset/pdf/LME.pdf",
             "asset/pdf/無신여성시대건강보험.pdf"
             ]

# Define keywords for each PDF
keywords = [
    ("abl여30세.pdf", "해약환급금 예시표"),
    ("LKB.pdf", "해약환급금 예시표"),
    ("LME.pdf", "해약환급금 예시표"),
    ("無신여성시대건강보험.pdf", "보험금의 종류 및 지급사유")
]


In [8]:
pdf_path =  pdf_paths[0]
print(pdf_path)
keyword = keywords[0]
print(keyword)

asset/pdf/abl여30세.pdf
('abl여30세.pdf', '해약환급금 예시표')


In [9]:
pdf_name = os.path.basename(pdf_path)

keyword = '해약환급금 예시표'

if keyword is not None:
    pages_with_keyword = find_keyword_pages(pdf_path, keyword)
    if pages_with_keyword:
        extract_and_save_tables(
            pdf_path, pages_with_keyword, keyword, output_folder)
        print(f"Extracted tables for {pdf_name} containing '{keyword}'")
    else:
        print(f"Keyword '{keyword}' not found in {pdf_name}")
else:
    print(f"No keyword defined for {pdf_name}")


Keyword '해약환급금 예시표' not found in abl여30세.pdf


In [ ]:
# Process each PDF
for pdf_path in pdf_paths:
    pdf_name = os.path.basename(pdf_path)
    keyword = next((k[1] for k in keywords if k[0] == pdf_name), None)

    if keyword is not None:
        pages_with_keyword = find_keyword_pages(pdf_path, keyword)
        if pages_with_keyword:
            extract_and_save_tables(
                pdf_path, pages_with_keyword, keyword, output_folder)
            print(f"Extracted tables for {pdf_name} containing '{keyword}'")
        else:
            print(f"Keyword '{keyword}' not found in {pdf_name}")
    else:
        print(f"No keyword defined for {pdf_name}")
